## Code structure:   
i want to adappt and create a subdataset for different tasks,
    this comprises: joining labels together, 
    filtering out some classes
    or selecting others only. 
    
    
1 - load dataset:
    structure is:
        segmentID, spectrogram, matrixlabel (9*259) each row corresponds to the following classes
                    
        1- Male inhale
        2- Female inhale
        3- Chick
        4- Male bout
        5- Female bout
        6- Flapping
        7- Male grunt/noise
        8- Female grunt/noise
        9- Unknown grunt/noise

2 - define manipulation to do
    
    
    


In [1]:
import pandas as pd
import numpy as np
import os
import librosa
from tqdm.notebook import tqdm
import csv


In [2]:
def remove_label(dataset,labels_dict, new_labels_dict  ):
    '''remove classes from label_matrix '''
    inve_new_label_dict = {v: k for k, v in new_labels_dict.items()}
    inv_labels_dic = {v: k for k, v in labels_dict.items()} 
    
    
    new_dataset = []
    for segm_id, spectro, label_matrix in dataset:
        new_label_matrix = np.zeros((len(new_labels_dict.keys()), 259))
        for lb, new_indx  in inve_new_label_dict.items():
            new_label_matrix[new_indx,:] = label_matrix[inv_labels_dic[lb]]
        
        new_dataset.append([segm_id, spectro, new_label_matrix])

    new_dataset = np.asarray(new_dataset)
    return new_dataset, new_labels_dict

def join_labels(labels_2_join, dataset, labels_dict, new_labels_dict, new_class_label):
    '''join information in different rows,   and create aditional row for joint labels
    '''
        
    inve_new_label_dict = {v: k for k, v in new_labels_dict.items()}
    inv_labels_dic = {v: k for k, v in labels_dict.items()}
    labels_2_join_indx = []
    for label in labels_2_join:
        labels_2_join_indx.append(inv_labels_dic[label])
        # based on labels_dict!

    new_dataset = []


    for segm_id, spectro, label_matrix in dataset:
        
        new_label_matrix = np.zeros((len(new_labels_dict.keys()), 259))
        new_label_matrix[0:label_matrix.shape[0], :] = label_matrix

        joined_labels_row = np.sum(label_matrix[labels_2_join_indx, :],0)
        #just normalizing to 1s and zeros...
        joined_labels_row = np.divide(joined_labels_row, joined_labels_row, out=np.zeros_like(joined_labels_row), where=joined_labels_row!=0)
        
        indx = inve_new_label_dict[new_class_label]
        new_label_matrix[int(indx),:] = joined_labels_row

        new_dataset.append([segm_id, spectro, new_label_matrix])

    new_dataset = np.asarray(new_dataset)
    return new_dataset, new_labels_dict


def select_examples_based_labels(dataset, positive_labels, labels_dict, mode='keepPosOnly' ):
    '''keepPosOnly: keep only examples that have ones in the given rows (positive_labels) 
    (remove everything else or the ones that have on;ly zeros!)
        
        keepNegOnly: keep only examples that have ones only onm other rows than the ones 
    defined as positive!
        
        keepSilencesOnly: select examples that have zeros through the whole matrix! 
    (becareful between the distinction of negs and silences depends on the dataset given!)
    
    '''        
    
    
    
    
    return sub_dataset
    



In [9]:
data_processed = '/home/ines/Dropbox/QMUL/PHD/manx_shearwaters/data/data_processed/'

labels_dict = {0:'male_in', 1:'female_in', 2:'chick', 3:'male_bout', 4:'female_bout', 5:'flapping', 6: 'male_grunt', 7:'female_grunt', 8:'unk_grunt' }

In [4]:
dataset = np.load(data_processed+'dataset_25_03.npy', allow_pickle=True)

In [10]:
# Adult_vs_chick_dataset:

# lets create dataset with the following 3 classes: Adult_bouts, chicks and adult_grunts

#0- get silences dataset
#1 - join female_bouts and male bouts into adult_bouts_class
# 2 - join the 3 grunt classes into adult_grunts (fem_grunt, male_grunt, unk_grunt)
# 3 - remove all other classes

# silences_dataset = select_examples_based_labels(adult_chick_dataset_joined)

#1
new_labels_dict_1 = {0:'male_in', 1:'female_in', 2:'chick', 3:'male_bout', 4:'female_bout', 5:'flapping', 6: 'male_grunt', 7:'female_grunt', 8:'unk_grunt', 9:'adult_bout' }
new_class_label =  'adult_bout'
adult_chick_dataset_1, new_labels_dict_1= join_labels(['male_bout', 'female_bout'], dataset, labels_dict, new_labels_dict_1, new_class_label)


#2
new_labels_dict_2 = {0:'male_in', 1:'female_in', 2:'chick', 3:'male_bout', 4:'female_bout', 5:'flapping', 6: 'male_grunt', 7:'female_grunt', 8:'unk_grunt', 9:'adult_bout' , 10: 'adult_grunt'}
new_class_label =  'adult_grunt'
adult_chick_dataset_2, new_labels_dict_2= join_labels(['male_grunt', 'female_grunt', 'unk_grunt'], adult_chick_dataset_1, new_labels_dict_1, new_labels_dict_2, new_class_label)

#3
new_labels_dict_3 = {0:'adult_grunt', 1:'adult_bout', 2:'chick', }
adult_chick_dataset_3, new_labels_dict_3= remove_label( adult_chick_dataset_2,new_labels_dict_2, new_labels_dict_3)



# adult_chick_dataset_1, new_labels_dict = join_labels([0,1], dataset, labels_dict)
# # this will have examples of adults chicks, plus all other classes (negatives) and silences

# adult_chick_dataset, new_labels_dict = remove_labels([2,3,4,5,6,7,8], adult_chick_dataset_joined, new_labels_dict)
# #this will have the same examples as before but only adults and cchicks are explicitly labeled . for binary classification!

# adult_chick_dataset_only_positive = select_examples_based_labels(adult_chick_dataset, [0,1])
# #this removes all examples of non chicks or non adults plus silences


# adult_chick_dataset_negs_all_only = select_examples_based_labels[adult_chick_dataset]
# # this removes all the examples of adults and chicks and leaves silences and negative classes examples
# # ??

# adult_chick_dataset_silences_only = select_examples_based_labels(adult_chick_dataset_joined)
# # this will only contain silences! (no negatives nor positives!)


In [11]:
# verify:

print(new_labels_dict_3)
print(adult_chick_dataset_3.shape)
print(adult_chick_dataset_3[0,-1].shape)


print(new_labels_dict_2)
print(adult_chick_dataset_2.shape)
print(adult_chick_dataset_2[0,-1].shape)

print(new_labels_dict_1)
print(adult_chick_dataset_1.shape)
print(adult_chick_dataset_1[0,-1].shape)

print(labels_dict)
print(dataset.shape)
print(dataset[0,-1].shape)

{0: 'adult_grunt', 1: 'adult_bout', 2: 'chick'}
(38049, 3)
(3, 259)
{0: 'male_in', 1: 'female_in', 2: 'chick', 3: 'male_bout', 4: 'female_bout', 5: 'flapping', 6: 'male_grunt', 7: 'female_grunt', 8: 'unk_grunt', 9: 'adult_bout', 10: 'adult_grunt'}
(38049, 3)
(11, 259)
{0: 'male_in', 1: 'female_in', 2: 'chick', 3: 'male_bout', 4: 'female_bout', 5: 'flapping', 6: 'male_grunt', 7: 'female_grunt', 8: 'unk_grunt', 9: 'adult_bout'}
(38049, 3)
(10, 259)
{0: 'male_in', 1: 'female_in', 2: 'chick', 3: 'male_bout', 4: 'female_bout', 5: 'flapping', 6: 'male_grunt', 7: 'female_grunt', 8: 'unk_grunt'}
(38049, 3)
(9, 259)


### TESTS:

In [5]:

dataset_test = dataset[0:10, :]

In [7]:
dataset_test[:,-1].shape

(10,)

In [6]:
for i in range(dataset_test[:,-1].shape[0]):
    dataset_test[:,-1][i][1, 0:100] = 1
    dataset_test[:,-1][i][4, 50:200] = 1


In [8]:
dataset_test[0,-1]

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
#Test functions!
new_labels_dict = labels_dict
new_labels_dict[9] = 'testing!'
new_class_label =  'testing!'
test_dataset, labels_dict_result = join_labels(['female_in', 'female_bout'], dataset_test, labels_dict, new_labels_dict, new_class_label)


new_labels_dict2 = {0:'testing!', 1:'male_bout', 2: 'female_in'}
test_dataset_removed, last_labels_dict = remove_label(test_dataset,new_labels_dict, new_labels_dict2  )

In [ ]:
print(test_dataset.shape)
print(test_dataset[0,-1].shape)
print(dataset_test[0,-1].shape)
print(labels_dict_result)
# print(dataset_test[0,-1][0])
# print(test_dataset[0,-1][0])

# print(dataset_test[0,-1][4])
print(test_dataset[0,-1][4])

# print(dataset_test[0,-1][1])
print(test_dataset[0,-1][1])

print('resu;lt joining rows 1 and 4:')
print(test_dataset[0,-1][-1])

In [ ]:
print(test_dataset_removed.shape)
print(test_dataset_removed[0,-1].shape)
print(dataset_test[0,-1].shape)
print(test_dataset[0,-1].shape)
print(last_labels_dict)
# print(dataset_test[0,-1][0])
# print(test_dataset[0,-1][0])

# print(dataset_test[0,-1][4])
print(test_dataset_removed[0,-1])

In [28]:
ex_label_mat = dataset[1741,2]
print(ex_label_mat)
print(ex_label_mat.shape)

new_row = np.sum(ex_label_mat[[1,2,3,4], :],0)
print(new_row)
print(new_row.shape)

new_row[0:5] = 5
print(new_row)

new_row = np.divide(new_row, new_row, out=np.zeros_like(new_row), where=new_row!=0)
print(new_row)


new_ex_label_mat = ex_label_mat
new_ex_label_mat = np.delete(new_ex_label_mat,[1,2,3,4], axis = 0)
print(new_ex_label_mat)
print(new_ex_label_mat.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(9, 259)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(259,)
[5. 5. 5. 5. 5. 0. 0. 0. 0. 0. 0. 0. 0.

In [ ]:
import pdb

dataset = [] # [dt_id, spectrogram, labelmatrix]

for audio_file in tqdm(os.listdir(audio_path), desc='load_audio'):
    print('load audio')
    dataset_per_file = []
    y, sr = librosa.load(audio_path+audio_file, sr=sample_rate)
    length = int(len(y) / sr) #seconds
    remainder = length % segment_size
    #     import pdb; pdb.set_trace()

    audio_id = audio_file[:-4]

    raw_annotation_file = audio_id  + '_a.csv'

    processed_annotations = process_raw_annotations_file(raw_annotation_file, annotations_path, labels_dict)    

    for t in tqdm(range(0, length - remainder - segment_size, slide),
                  desc='create_spectros'):
        start = t
    #         print('start', str(start))
        stop = t + segment_size

        print('start in frames', str(sr*start))
        print('stop in frames: ', str((sr*stop)))

        current_y = y[sr*start:(sr*stop)]
        # create spectrogram
        spectro = librosa.feature.melspectrogram(y=current_y, sr=sr, n_mels=mels,
                                                 fmax=sr/2)

        dt_id = audio_id + '_' + str(start) + 's_to' + str(stop) + 's'
        print(dt_id)

        label_matrix = get_label_matrix_per_segment(start, stop, sr, timesteps, timesteps_per_second, processed_annotations, labels_dict )
        dataset_per_file.append([dt_id, spectro, label_matrix])
        dataset.append([dt_id, spectro, label_matrix])
        
    
    #save data processed for this file
    d_array = np.asarray(dataset_per_file)
    np.save(save_path+audio_id, d_array)

data_array = np.asarray(dataset)
np.save(save_path+'data_25_03', data_array)


In [ ]:
int(np.floor(1.5))

In [1]:
import pandas as pd
import numpy as np
import os
import librosa
from tqdm.notebook import tqdm
import csv


In [2]:
dataset = []
for file in os.listdir('/home/ines/Dropbox/QMUL/PHD/manx_shearwaters/data/data_processed/'):
#     print(file)
    data = np.load('/home/ines/Dropbox/QMUL/PHD/manx_shearwaters/data/data_processed/' + file, allow_pickle=True)
#     print(data.shape)
    dataset.extend(data)
        

In [3]:
dataset_array = np.asarray(dataset)

In [4]:
dataset_array.shape

(38049, 3)

In [ ]:
np.save('/home/ines/Dropbox/QMUL/PHD/manx_shearwaters/data/data_processed/dataset_25_03.npy', dataset_array)